In [1]:
import numpy as np
import pylab as plt
import swyft
import torch
from scipy import stats
%load_ext autoreload
%autoreload 2

np.random.seed(25)
torch.manual_seed(25)

In [2]:
DEVICE = 'cuda:0'
MAX_EPOCHS = 100

## Torus model

In [3]:
def simulator(z, center = np.array([0.6, 0.8])):
    r = sum((z[:2]-center)**2)**0.5  # Return radial distance from center
    x = np.array([z[0], r, z[2]])
    return x

def noise(x, z = None, noise = np.array([0.03, 0.005, 0.2])):
    n = np.random.randn(*x.shape)*noise
    return x + n

z0 = np.array([0.57, 0.8, 1.0])
zdim = len(z0)
x0 = simulator(z0)  # Using Asimov data

In [4]:
cache = swyft.MemoryCache(zdim = zdim, xshape = x0.shape)

Creating new cache.


## Inference

In [5]:
intensity = swyft.get_unit_intensity(expected_n=10000, dim=zdim)
cache._grow(intensity)
cache.simulate(simulator)
points = swyft.Points(cache, intensity, noise)

Simulate:   2%|▏         | 223/10130 [00:00<00:04, 2224.49it/s]

Adding 10130 new samples. Run simulator!


Simulate: 100%|██████████| 10130/10130 [00:03<00:00, 3007.92it/s]


In [6]:
re = swyft.RatioEstimator(x0, points, device=DEVICE)

In [7]:
re.train(max_epochs = 10, batch_size = 32, lr_schedule = [1e-3, 3e-4, 1e-4])

Initializing network...
yshape (shape of features between head and legs): 1
Start training
No new simulator runs required.
LR iteration 0
torch.Size([64, 3]) torch.Size([64, 3, 1])
torch.Size([64, 3, 4])
torch.Size([3, 256, 2]) torch.Size([3, 256]) torch.Size([64, 3, 4])
torch.Size([64, 3, 4, 1])


RuntimeError: invalid argument 6: wrong matrix size at /opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/THC/generic/THCTensorMathBlas.cu:558

In [ ]:
re_prev = None
for r in range(4):
    td = swyft.TrainData(x0 = x0, zdim = zdim, noisehook = noise, cache = ds, nsamples = 10000, threshold = 1e-4, parent = re_prev)
    ds.simulate(simulator)
    re = swyft.RatioEstimator(zdim, td, device = DEVICE, combinations = [[0], [1], [2]])
    re.train(max_epochs = MAX_EPOCHS, nbatch = 32, lr_schedule = [1e-3, 3e-4, 1e-4])
    re_prev = re

In [ ]:
swyft.plot1d(re, x0 = x0, z0 = z0, cmap = 'Greys', dims = (12, 4))

In [ ]:
re2 = swyft.RatioEstimator(zdim, td, device = DEVICE, combinations = [[0, 1], [0, 2], [1, 2]])

In [ ]:
re2.train(max_epochs = MAX_EPOCHS, nbatch = 32, lr_schedule = [1e-3, 3e-4, 1e-4])

In [ ]:
swyft.corner(re, re2, x0 = x0, z0 = z0, cmap = 'Greys', dim = 10)